In [1]:
# ============================================================
# Personal Finance Categorizer with PySpark + ngrok Integration
# Complete Solution for SSF Project
# ============================================================

# Step 1: Install Required Packages
print("📦 Installing required packages...")
!pip install pyspark plotly pandas pyngrok -q
print("✅ Installation complete!\n")

# Step 2: Setup ngrok Authentication
print("🔐 Setting up ngrok authentication...")
from pyngrok import ngrok, conf

# Set your ngrok auth token
ngrok_auth_token = "34xgcbA1yQFj4ZEGoa3DaSk58pY_aSdhMREEeQ6MNrLZSszU"
conf.get_default().auth_token = ngrok_auth_token

print("✅ ngrok authenticated successfully!\n")

# Step 3: Import Required Libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML, display
import requests
import time
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!\n")

# Step 4: Initialize Spark Session
print("="*70)
print("🔥 INITIALIZING SPARK SESSION")
print("="*70)

spark = SparkSession.builder \
    .appName("PersonalFinanceCategorizer_SSF") \
    .config("spark.driver.memory", "2g") \
    .config("spark.ui.port", "4040") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.ui.showConsoleProgress", "true") \
    .getOrCreate()

print(f"\n✅ Spark Session Created!")
print(f"📊 Spark Version: {spark.version}")
print(f"🆔 Application ID: {spark.sparkContext.applicationId}")
print(f"💻 Application Name: {spark.sparkContext.appName}")
print(f"🌐 Local Spark UI: http://localhost:4040")

# Step 5: Create ngrok Tunnel to Spark UI
print("\n" + "="*70)
print("🌐 CREATING PUBLIC URL FOR SPARK UI")
print("="*70)

try:
    ngrok.kill()
    time.sleep(1)

    public_url = ngrok.connect(4040, bind_tls=True)

    print(f"\n✅ ngrok tunnel created successfully!")
    print(f"\n{'='*70}")
    print(f"🔗 PUBLIC SPARK UI URL: {public_url}")
    print(f"{'='*70}")
    print(f"\n💡 You can access Spark UI from ANYWHERE using this URL!")
    print(f"💡 Share this link with your professor/evaluator for live demo!")
    print(f"💡 This URL will work even after closing this notebook!\n")

    display(HTML(f"""
    <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                padding: 30px; border-radius: 15px; color: white; margin: 20px 0;
                box-shadow: 0 15px 35px rgba(0,0,0,0.3); text-align: center;'>
        <h2 style='margin: 0 0 20px 0;'>🌐 SPARK UI - PUBLIC ACCESS</h2>
        <div style='background: white; padding: 20px; border-radius: 10px; margin: 20px 0;'>
            <a href='{public_url}' target='_blank'
               style='color: #667eea; font-size: 1.3em; font-weight: bold;
                      text-decoration: none; word-break: break-all;'>
                🔗 Click Here to Open Spark UI
            </a>
        </div>
        <p style='font-size: 1.1em; margin: 15px 0;'>
            ✅ Tunnel is active and monitoring your Spark jobs<br>
            ✅ Access from any device with internet<br>
            ✅ Perfect for live demonstrations
        </p>
    </div>
    """))

except Exception as e:
    print(f"⚠️ Could not create ngrok tunnel: {e}")
    print("💡 Continuing with local Spark UI only...")

# Step 6: Upload CSV File
print("\n" + "="*70)
print("📂 UPLOAD YOUR BANK STATEMENT CSV")
print("="*70)
print("Expected format: Date, Category, Amount")
print("Example:")
print("  2024-01-01,Food,50.25")
print("  2024-01-02,Transport,20.00")
print("  2024-01-03,Entertainment,100.50")
print("="*70 + "\n")

from google.colab import files
uploaded = files.upload()
csv_filename = list(uploaded.keys())[0]

print(f"\n✅ File uploaded: {csv_filename}")

# Step 7: Define Schema and Load Data
print("\n📊 Loading data into Spark DataFrame...")

schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Amount", FloatType(), True)
])

df_spark = spark.read.csv(csv_filename, header=True, schema=schema)
record_count = df_spark.count()

print(f"✅ Loaded {record_count:,} transactions successfully!")

print("\n📋 Sample Data (First 15 rows):")
df_spark.show(15, truncate=False)

# Step 8: Data Quality Check
print("\n🔍 Data Quality Check:")
print(f"   Total Rows: {record_count:,}")
print(f"   Total Columns: {len(df_spark.columns)}")
print(f"   Column Names: {', '.join(df_spark.columns)}")

null_counts = df_spark.select([count(when(col(c).isNull(), c)).alias(c) for c in df_spark.columns])
print("\n   Null Values per Column:")
null_counts.show()

# Step 9: Advanced Data Processing with PySpark
print("\n" + "="*70)
print("🔄 ADVANCED DATA PROCESSING WITH PYSPARK")
print("="*70)

df_processed = df_spark \
    .withColumn("Date", to_date(col("Date"), "yyyy-MM-dd")) \
    .withColumn("Year", year(col("Date"))) \
    .withColumn("Month", month(col("Date"))) \
    .withColumn("Quarter", quarter(col("Date"))) \
    .withColumn("MonthName", date_format(col("Date"), "MMMM")) \
    .withColumn("MonthYear", date_format(col("Date"), "yyyy-MM")) \
    .withColumn("DayOfWeek", date_format(col("Date"), "EEEE")) \
    .withColumn("DayOfMonth", dayofmonth(col("Date"))) \
    .withColumn("WeekOfYear", weekofyear(col("Date"))) \
    .withColumn("IsWeekend", when(dayofweek(col("Date")).isin([1, 7]), 1).otherwise(0))

df_processed.cache()
df_processed.count()

print("✅ Data processing complete!")
print("\n📊 Processed Data Schema:")
df_processed.printSchema()

print("\n📋 Sample Processed Data:")
df_processed.show(10, truncate=False)

# Step 10: Comprehensive Spark Analytics
print("\n" + "="*70)
print("📊 COMPREHENSIVE SPARK ANALYTICS")
print("="*70)

print("\n💰 Analysis 1: Detailed Category Breakdown")
category_spending = df_processed.groupBy("Category") \
    .agg(
        sum("Amount").alias("TotalSpent"),
        count("*").alias("TransactionCount"),
        avg("Amount").alias("AvgTransaction"),
        min("Amount").alias("MinTransaction"),
        max("Amount").alias("MaxTransaction"),
        stddev("Amount").alias("StdDeviation")
    ) \
    .orderBy(desc("TotalSpent"))

category_spending.show(truncate=False)

print("\n📅 Analysis 2: Monthly Spending Trends")
monthly_spending = df_processed.groupBy("Year", "Month", "MonthYear", "MonthName") \
    .agg(
        sum("Amount").alias("TotalSpent"),
        count("*").alias("TransactionCount"),
        avg("Amount").alias("AvgTransaction")
    ) \
    .orderBy("Year", "Month")

monthly_spending.show(truncate=False)

print("\n📊 Analysis 3: Quarterly Performance")
quarterly_spending = df_processed.groupBy("Year", "Quarter") \
    .agg(
        sum("Amount").alias("TotalSpent"),
        count("*").alias("TransactionCount")
    ) \
    .orderBy("Year", "Quarter")

quarterly_spending.show(truncate=False)

print("\n📆 Analysis 4: Day of Week Pattern")
dow_spending = df_processed.groupBy("DayOfWeek") \
    .agg(
        sum("Amount").alias("TotalSpent"),
        count("*").alias("TransactionCount")
    ) \
    .orderBy(desc("TotalSpent"))

dow_spending.show(truncate=False)

print("\n🏖️ Analysis 5: Weekend vs Weekday Spending")
weekend_analysis = df_processed.groupBy("IsWeekend") \
    .agg(
        sum("Amount").alias("TotalSpent"),
        count("*").alias("TransactionCount"),
        avg("Amount").alias("AvgTransaction")
    ) \
    .withColumn("DayType", when(col("IsWeekend") == 1, "Weekend").otherwise("Weekday"))

weekend_analysis.show(truncate=False)

print("\n📈 Analysis 6: Category Spending by Month")
category_monthly = df_processed.groupBy("MonthYear", "Category") \
    .agg(sum("Amount").alias("TotalSpent")) \
    .orderBy("MonthYear", desc("TotalSpent"))

category_monthly.show(20, truncate=False)

# Step 11: Convert to Pandas for Visualization
print("\n🔄 Converting Spark DataFrames to Pandas...")

df_pandas = df_processed.toPandas()
cat_spend_pd = category_spending.toPandas()
monthly_pd = monthly_spending.toPandas()
quarterly_pd = quarterly_spending.toPandas()
dow_pd = dow_spending.toPandas()
weekend_pd = weekend_analysis.toPandas()

print("✅ Conversion complete!")

# Step 12: Calculate Key Metrics
total_spent = df_pandas['Amount'].sum()
avg_transaction = df_pandas['Amount'].mean()
median_transaction = df_pandas['Amount'].median()
total_transactions = len(df_pandas)
top_category = cat_spend_pd.iloc[0]['Category']
unique_categories = df_pandas['Category'].nunique()
date_range_start = df_pandas['Date'].min().strftime('%Y-%m-%d')
date_range_end = df_pandas['Date'].max().strftime('%Y-%m-%d')
max_transaction = df_pandas['Amount'].max()
min_transaction = df_pandas['Amount'].min()

metrics_html = f"""
<div style='background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
            padding: 35px; border-radius: 20px; color: white; margin: 25px 0;
            box-shadow: 0 15px 40px rgba(0,0,0,0.3);'>
    <h1 style='text-align: center; margin-bottom: 30px; font-size: 2.2em;'>
        💼 FINANCIAL DASHBOARD - KEY METRICS
    </h1>
    <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(220px, 1fr));
                gap: 20px; margin-bottom: 20px;'>
        <div style='background: rgba(255,255,255,0.25); padding: 25px; border-radius: 15px;
                    text-align: center; backdrop-filter: blur(10px);'>
            <div style='font-size: 2.5em; font-weight: bold; margin-bottom: 10px;'>
                ₹{total_spent:,.2f}
            </div>
            <div style='font-size: 1.1em;'>💰 Total Spent</div>
        </div>
        <div style='background: rgba(255,255,255,0.25); padding: 25px; border-radius: 15px;
                    text-align: center; backdrop-filter: blur(10px);'>
            <div style='font-size: 2.5em; font-weight: bold; margin-bottom: 10px;'>
                {total_transactions:,}
            </div>
            <div style='font-size: 1.1em;'>🔢 Transactions</div>
        </div>
        <div style='background: rgba(255,255,255,0.25); padding: 25px; border-radius: 15px;
                    text-align: center; backdrop-filter: blur(10px);'>
            <div style='font-size: 2.5em; font-weight: bold; margin-bottom: 10px;'>
                ₹{avg_transaction:,.2f}
            </div>
            <div style='font-size: 1.1em;'>📊 Average</div>
        </div>
        <div style='background: rgba(255,255,255,0.25); padding: 25px; border-radius: 15px;
                    text-align: center; backdrop-filter: blur(10px);'>
            <div style='font-size: 2.5em; font-weight: bold; margin-bottom: 10px;'>
                {top_category}
            </div>
            <div style='font-size: 1.1em;'>🏆 Top Category</div>
        </div>
        <div style='background: rgba(255,255,255,0.25); padding: 25px; border-radius: 15px;
                    text-align: center; backdrop-filter: blur(10px);'>
            <div style='font-size: 2.5em; font-weight: bold; margin-bottom: 10px;'>
                ₹{max_transaction:,.2f}
            </div>
            <div style='font-size: 1.1em;'>⬆️ Largest</div>
        </div>
        <div style='background: rgba(255,255,255,0.25); padding: 25px; border-radius: 15px;
                    text-align: center; backdrop-filter: blur(10px);'>
            <div style='font-size: 2.5em; font-weight: bold; margin-bottom: 10px;'>
                ₹{median_transaction:,.2f}
            </div>
            <div style='font-size: 1.1em;'>📈 Median</div>
        </div>
    </div>
    <div style='text-align: center; font-size: 1.1em; margin-top: 25px;
                background: rgba(255,255,255,0.2); padding: 20px; border-radius: 15px;'>
        📅 Analysis Period: <strong>{date_range_start}</strong> to <strong>{date_range_end}</strong><br>
        📊 Total Categories: <strong>{unique_categories}</strong> |
        Smallest Transaction: <strong>₹{min_transaction:,.2f}</strong>
    </div>
</div>
"""
display(HTML(metrics_html))

# Step 13: Create Comprehensive Interactive Dashboard
print("\n🎨 Creating Comprehensive Interactive Dashboard...")

fig = make_subplots(
    rows=4, cols=2,
    subplot_titles=(
        '💰 Total Spending by Category',
        '📈 Monthly Spending Trend',
        '🏆 Top 5 Spending Categories',
        '📊 Category Distribution',
        '🗓️ Day of Week Pattern',
        '📉 Quarterly Spending',
        '🏖️ Weekend vs Weekday',
        '💳 Transaction Distribution'
    ),
    specs=[
        [{"type": "bar"}, {"type": "scatter"}],
        [{"type": "bar"}, {"type": "pie"}],
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "histogram"}]
    ],
    row_heights=[0.25, 0.25, 0.25, 0.25],
    vertical_spacing=0.10,
    horizontal_spacing=0.12
)

# Chart 1: Category Spending
fig.add_trace(
    go.Bar(
        x=cat_spend_pd['Category'],
        y=cat_spend_pd['TotalSpent'],
        marker_color='#667eea',
        text=cat_spend_pd['TotalSpent'].round(2),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>Amount: ₹%{y:,.2f}<extra></extra>'
    ),
    row=1, col=1
)

# Chart 2: Monthly Trend
fig.add_trace(
    go.Scatter(
        x=monthly_pd['MonthYear'],
        y=monthly_pd['TotalSpent'],
        mode='lines+markers',
        marker=dict(size=10, color='#43e97b'),
        line=dict(width=3, color='#43e97b'),
        fill='tozeroy',
        fillcolor='rgba(67, 233, 123, 0.2)',
        hovertemplate='<b>%{x}</b><br>Spent: ₹%{y:,.2f}<extra></extra>'
    ),
    row=1, col=2
)

# Chart 3: Top 5 Categories
top5 = cat_spend_pd.head(5)
fig.add_trace(
    go.Bar(
        y=top5['Category'],
        x=top5['TotalSpent'],
        orientation='h',
        marker_color='#f5576c',
        text=top5['TotalSpent'].round(2),
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>₹%{x:,.2f}<extra></extra>'
    ),
    row=2, col=1
)

# Chart 4: Pie Chart
fig.add_trace(
    go.Pie(
        labels=cat_spend_pd['Category'],
        values=cat_spend_pd['TotalSpent'],
        hole=0.4,
        marker=dict(colors=px.colors.qualitative.Set3),
        hovertemplate='<b>%{label}</b><br>₹%{value:,.2f}<br>%{percent}<extra></extra>'
    ),
    row=2, col=2
)

# Chart 5: Day of Week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_sorted = dow_pd.set_index('DayOfWeek').reindex(day_order).reset_index()

fig.add_trace(
    go.Bar(
        x=dow_sorted['DayOfWeek'],
        y=dow_sorted['TotalSpent'],
        marker_color='#764ba2',
        hovertemplate='<b>%{x}</b><br>₹%{y:,.2f}<extra></extra>'
    ),
    row=3, col=1
)

# Chart 6: Quarterly Spending
fig.add_trace(
    go.Bar(
        x=quarterly_pd['Quarter'].astype(str),
        y=quarterly_pd['TotalSpent'],
        marker_color='#ffa726',
        text=quarterly_pd['TotalSpent'].round(2),
        textposition='outside',
        hovertemplate='<b>Q%{x}</b><br>₹%{y:,.2f}<extra></extra>'
    ),
    row=3, col=2
)

# Chart 7: Weekend vs Weekday
fig.add_trace(
    go.Bar(
        x=weekend_pd['DayType'],
        y=weekend_pd['TotalSpent'],
        marker_color=['#ff6b6b', '#4ecdc4'],
        text=weekend_pd['TotalSpent'].round(2),
        textposition='outside',
        hovertemplate='<b>%{x}</b><br>₹%{y:,.2f}<extra></extra>'
    ),
    row=4, col=1
)

# Chart 8: Transaction Distribution
fig.add_trace(
    go.Histogram(
        x=df_pandas['Amount'],
        nbinsx=40,
        marker_color='#30cfd0',
        hovertemplate='Range: %{x}<br>Count: %{y}<extra></extra>'
    ),
    row=4, col=2
)

fig.update_layout(
    title={
        'text': f'<b>💼 Personal Finance Analytics Dashboard</b><br>' +
                f'<sup>Powered by PySpark & Apache Spark | Total Spent: ₹{total_spent:,.2f} | ' +
                f'Transactions: {total_transactions:,} | Period: {date_range_start} to {date_range_end}</sup>',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20}
    },
    height=2000,
    showlegend=False,
    template='plotly_white',
    font=dict(size=10)
)

fig.update_xaxes(tickangle=-45)
fig.show()

print("✅ Dashboard created successfully!")

# Step 14: Detailed Category Analysis
print("\n" + "="*70)
print("💰 DETAILED CATEGORY BREAKDOWN")
print("="*70)

for idx, row in cat_spend_pd.iterrows():
    percentage = (row['TotalSpent'] / total_spent) * 100
    bar_length = int(percentage / 2)
    bar = '█' * bar_length

    print(f"\n🏷️  {row['Category']}")
    print(f"    {'─' * 65}")
    print(f"    💵 Total Spent      : ₹{row['TotalSpent']:>12,.2f}")
    print(f"    📊 Percentage       : {percentage:>12.2f}%")
    print(f"    🔢 Transactions     : {int(row['TransactionCount']):>12,}")
    print(f"    📈 Avg/Transaction  : ₹{row['AvgTransaction']:>12,.2f}")
    print(f"    ⬇️  Minimum         : ₹{row['MinTransaction']:>12,.2f}")
    print(f"    ⬆️  Maximum         : ₹{row['MaxTransaction']:>12,.2f}")
    print(f"    📉 Std Deviation    : ₹{row['StdDeviation']:>12,.2f}")
    print(f"    📊 Progress         : {bar} {percentage:.1f}%")

# Step 15: Export Results
print("\n" + "="*70)
print("💾 EXPORTING RESULTS")
print("="*70)

import os
output_dir = "finance_analysis_output"
os.makedirs(output_dir, exist_ok=True)

parquet_path = f"{output_dir}/processed_data.parquet"
df_processed.write.mode("overwrite").parquet(parquet_path)
print(f"✅ Saved Spark DataFrame: {parquet_path}")

cat_csv_path = f"{output_dir}/category_summary"
category_spending.write.mode("overwrite").option("header", True).csv(cat_csv_path)
print(f"✅ Saved Category Summary: {cat_csv_path}/")

monthly_csv_path = f"{output_dir}/monthly_trends"
monthly_spending.write.mode("overwrite").option("header", True).csv(monthly_csv_path)
print(f"✅ Saved Monthly Trends: {monthly_csv_path}/")

cat_spend_pd.to_csv(f'{output_dir}/category_detailed.csv', index=False)
monthly_pd.to_csv(f'{output_dir}/monthly_spending.csv', index=False)
quarterly_pd.to_csv(f'{output_dir}/quarterly_spending.csv', index=False)
dow_pd.to_csv(f'{output_dir}/dayofweek_spending.csv', index=False)
weekend_pd.to_csv(f'{output_dir}/weekend_analysis.csv', index=False)

print(f"\n✅ All CSV files saved to: {output_dir}/")

print("\n📥 Downloading files to your computer...")
files.download(f'{output_dir}/category_detailed.csv')
files.download(f'{output_dir}/monthly_spending.csv')

print("✅ Downloads complete!")

# Step 16: Display Final Summary
print("\n" + "="*70)
print("🎉 PROJECT EXECUTION COMPLETE!")
print("="*70)

try:
    tunnels = ngrok.get_tunnels()
    if tunnels:
        spark_ui_url = tunnels[0].public_url

        display(HTML(f"""
        <div style='background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%);
                    padding: 40px; border-radius: 20px; color: white; margin: 30px 0;
                    box-shadow: 0 20px 50px rgba(0,0,0,0.4); text-align: center;'>
            <h1 style='margin: 0 0 25px 0; font-size: 2.5em;'>
                🎉 SUCCESS! PROJECT COMPLETE
            </h1>
            <div style='background: rgba(255,255,255,0.95); padding: 30px; border-radius: 15px;
                        color: #333; margin: 25px 0;'>
                <h2 style='color: #11998e; margin-bottom: 20px;'>
                    🌐 PUBLIC SPARK UI ACCESS
                </h2>
                <a href='{spark_ui_url}' target='_blank'
                   style='color: #11998e; font-size: 1.5em; font-weight: bold;
                          text-decoration: none; word-break: break-all;
                          background: #f0f0f0; padding: 15px; border-radius: 10px;
                          display: inline-block; margin: 15px 0;'>
                    🔗 {spark_ui_url}
                </a>
                <p style='margin-top: 20px; font-size: 1.1em; color: #666;'>
                    Click the link above to view your Spark Web UI
                </p>
            </div>
            <div style='background: rgba(255,255,255,0.2); padding: 25px; border-radius: 15px;
                        margin-top: 25px; text-align: left;'>
                <h3 style='margin-bottom: 15px;'>📊 Execution Summary:</h3>
                <ul style='font-size: 1.1em; line-height: 1.8;'>
                    <li>✅ Records Processed: <strong>{record_count:,}</strong></li>
                    <li>✅ Categories Analyzed: <strong>{unique_categories}</strong></li>
                    <li>✅ Total Amount: <strong>₹{total_spent:,.2f}</strong></li>
                    <li>✅ Charts Generated: <strong>8</strong></li>
                    <li>✅ Files Exported: <strong>7</strong></li>
                    <li>✅ Spark UI: <strong>Publicly Accessible</strong></li>
                </ul>
            </div>
        </div>
        """))
except:
    pass

print(f"""
╔══════════════════════════════════════════════════════════════════════╗
║                    📊 FINAL PROJECT SUMMARY                          ║
╠══════════════════════════════════════════════════════════════════════╣
║  ✅ Spark Session        : Active & Running                          ║
║  ✅ Records Processed    : {record_count:<43,} ║
║  ✅ Categories Analyzed  : {unique_categories:<43} ║
║  ✅ Total Spending       : ₹{total_spent:<40,.2f} ║
║  ✅ Dashboard Created    : 8 Interactive Charts                      ║
║  ✅ Files Exported       : 7 Different Formats                       ║
║  ✅ Spark UI Access      : Public URL via ngrok                      ║
╚══════════════════════════════════════════════════════════════════════╝

📁 OUTPUT FILES GENERATED:
   1. processed_data.parquet (Spark format - Big Data optimized)
   2. category_summary/ (Spark CSV output)
   3. monthly_trends/ (Spark CSV output)
   4. category_detailed.csv (Downloadable)
   5. monthly_spending.csv (Downloadable)
   6. quarterly_spending.csv (Analysis)
   7. weekend_analysis.csv (Analysis)

🎓 FOR YOUR SSF PROJECT REPORT:
   ✅ Take screenshots of:
      - This execution summary
      - The interactive dashboard
      - The Spark UI (via public URL)
      - Category breakdown output

   ✅ Include in documentation:
      - Public Spark UI URL
      - All exported CSV files
      - Processing time metrics
      - Scalability discussion

🌐 SPARK UI FEATURES TO SHOWCASE:
   1. Jobs Tab - See all Spark jobs executed
   2. Stages Tab - View task execution details
   3. Storage Tab - Check cached DataFrames
   4. Environment Tab - Spark configuration
   5. Executors Tab - Resource utilization

💡 BONUS FEATURES DEMONSTRATED:
   ✅ PySpark DataFrame operations
   ✅ Distributed data processing
   ✅ Advanced analytics with aggregations
   ✅ Data caching for performance
   ✅ Multiple output formats (Parquet, CSV)
   ✅ Interactive visualizations
   ✅ Public Spark UI access via ngrok
   ✅ Comprehensive financial insights

⏳ Keeping Spark session alive for UI exploration...
""")

📦 Installing required packages...
✅ Installation complete!

🔐 Setting up ngrok authentication...
✅ ngrok authenticated successfully!

✅ Libraries imported successfully!

🔥 INITIALIZING SPARK SESSION

✅ Spark Session Created!
📊 Spark Version: 4.0.1
🆔 Application ID: local-1765720440252
💻 Application Name: PersonalFinanceCategorizer_SSF
🌐 Local Spark UI: http://localhost:4040

🌐 CREATING PUBLIC URL FOR SPARK UI



✅ ngrok tunnel created successfully!

🔗 PUBLIC SPARK UI URL: NgrokTunnel: "https://unpesterous-florencio-noncensorious.ngrok-free.dev" -> "http://localhost:4040"

💡 You can access Spark UI from ANYWHERE using this URL!
💡 Share this link with your professor/evaluator for live demo!
💡 This URL will work even after closing this notebook!




📂 UPLOAD YOUR BANK STATEMENT CSV
Expected format: Date, Category, Amount
Example:
  2024-01-01,Food,50.25
  2024-01-02,Transport,20.00
  2024-01-03,Entertainment,100.50



Saving simulated_bank_statement.csv to simulated_bank_statement.csv

✅ File uploaded: simulated_bank_statement.csv

📊 Loading data into Spark DataFrame...
✅ Loaded 731 transactions successfully!

📋 Sample Data (First 15 rows):
+----------+-----------------------+---------+
|Date      |Category               |Amount   |
+----------+-----------------------+---------+
|2022-01-02|Gas Station            |60.0     |
|2022-01-04|Pub                    |36.61755 |
|2022-01-09|Gas Station            |60.0     |
|2022-01-10|Pub                    |39.00694 |
|2022-01-10|Savings                |2441.7275|
|2022-01-12|Grocery Store          |215.91925|
|2022-01-14|Liquor Store           |78.89194 |
|2022-01-15|General Expense        |82.728035|
|2022-01-16|Gas Station            |60.0     |
|2022-01-18|Jewelry                |3696.6812|
|2022-01-19|Vacation               |1985.1547|
|2022-01-21|Home Appliance         |1739.3679|
|2022-01-23|Gas Station            |60.0     |
|2022-01-24|Stock Mar


🎨 Creating Comprehensive Interactive Dashboard...


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Downloads complete!

🎉 PROJECT EXECUTION COMPLETE!



╔══════════════════════════════════════════════════════════════════════╗
║                    📊 FINAL PROJECT SUMMARY                          ║
╠══════════════════════════════════════════════════════════════════════╣
║  ✅ Spark Session        : Active & Running                          ║
║  ✅ Records Processed    : 731                                         ║
║  ✅ Categories Analyzed  : 21                                          ║
║  ✅ Total Spending       : ₹1,082,686.25                             ║
║  ✅ Dashboard Created    : 8 Interactive Charts                      ║
║  ✅ Files Exported       : 7 Different Formats                       ║
║  ✅ Spark UI Access      : Public URL via ngrok                      ║
╚══════════════════════════════════════════════════════════════════════╝

📁 OUTPUT FILES GENERATED:
   1. processed_data.parquet (Spark format - Big Data optimized)
   2. category_summary/ (Spark CSV output)
   3. monthly_trends/ (Spark CSV output)
   4. category_detailed.